In [5]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import cvxpy as cp
from scipy.stats import poisson, uniform, expon, pareto
from scipy.optimize import minimize
from tqdm import tqdm
from mdptoolbox import mdp, util
import itertools
from scipy.sparse import csr_matrix, lil_matrix
from matplotlib.patches import Patch
import math
import random
import sympy as sp
from sympy.printing.latex import print_latex

# Symbolic for derivatives

In [6]:
vi, xi = sp.symbols('v_i x_i')
a      = sp.symbols('a')
y      = sp.symbols('y')


t1 = vi*xi*a * (xi**(1-1/a) - xi**(2-1/a))
t2 = sp.integrate(vi*y*((a-1)*y**(-1/a)-(2*a-1)*y**(1-1/a)), (y, 0, xi))
uhat = t1 - t2

In [9]:
uhat.evalf(subs={a:1/2+0.1})
uhat

a*v_i*x_i*(x_i**(1 - 1/a) - x_i**(2 - 1/a)) - Piecewise((v_i*(log(x_i)/3 + 2/(3*x_i)) - oo*sign(v_i), Eq(a, 1/3)), (-v_i*log(x_i)/2 - oo*sign(v_i), Eq(a, 1/2)), (v_i*(-4*a**3*x_i**2*x_i**(1/a)*x_i**(1 - 1/a)/(6*a**2*x_i**(1/a) - 5*a*x_i**(1/a) + x_i**(1/a)) + 3*a**3*x_i**2/(6*a**2*x_i**(1/a) - 5*a*x_i**(1/a) + x_i**(1/a)) + 4*a**2*x_i**2*x_i**(1/a)*x_i**(1 - 1/a)/(6*a**2*x_i**(1/a) - 5*a*x_i**(1/a) + x_i**(1/a)) - 4*a**2*x_i**2/(6*a**2*x_i**(1/a) - 5*a*x_i**(1/a) + x_i**(1/a)) - a*x_i**2*x_i**(1/a)*x_i**(1 - 1/a)/(6*a**2*x_i**(1/a) - 5*a*x_i**(1/a) + x_i**(1/a)) + a*x_i**2/(6*a**2*x_i**(1/a) - 5*a*x_i**(1/a) + x_i**(1/a))), True))

In [10]:
sp.simplify(uhat.evalf(subs={a:1}))

v_i*x_i*(1.0 - 0.5*x_i)

In [11]:
sp.simplify(uhat.evalf(subs={a:2/3}))

v_i*(1.33333333333333*x_i**0.5 - 0.444444444444445*x_i**1.5)

In [166]:
print_latex(sp.simplify(uhat.evalf(subs={a:2/3, vi:2})))

2.66666666666667 x_{i}^{0.5} - 0.88888888888889 x_{i}^{1.5}


In [172]:
sp.simplify(uhat.evalf(subs={a:2/3, vi:4})+uhat.evalf(subs={a:2/3, vi:2}))

8.0*x_i**0.5 - 2.66666666666667*x_i**1.5

In [167]:
8/3

2.6666666666666665

In [146]:
# verify sol'n with standard FOC
v1, v2 = sp.symbols('v_1 v_2')
b1, b2 = sp.symbols('b_1 b_2')
a      = sp.symbols('a')

u1 = v1*b1**a / (b1**a + b2**a) - b1
u2 = v2*b2**a / (b1**a + b2**a) - b2
u1

-b_1 + b_1**a*v_1/(b_1**a + b_2**a)

In [158]:
u1.evalf(subs={a:2/3, v1:4})

4.0*b_1**0.666666666666667/(b_1**0.666666666666667 + b_2**0.666666666666667) - b_1

In [174]:
sp.diff(u1.evalf(subs={a:1, v1:4}), b1)

-4.0*b_1/(b_1 + b_2)**2 - 1 + 4.0/(b_1 + b_2)

In [179]:
foc1 = sp.simplify(sp.Eq(sp.diff(u1.evalf(subs={a:2/3}), b1), 0))
foc2 = sp.simplify(sp.Eq(sp.diff(u2.evalf(subs={a:2/3}), b2), 0))
foc1

Eq(0.666666666666667*b_2**0.666666666666667*v_1/(b_1**0.333333333333333*(b_1**0.666666666666667 + b_2**0.666666666666667)**2) - 1, 0)

In [180]:
foc2

Eq(0.666666666666667*b_1**0.666666666666667*v_2/(b_2**0.333333333333333*(b_1**0.666666666666667 + b_2**0.666666666666667)**2) - 1, 0)

In [181]:
sol = sp.solve((foc1, foc2), (b1, b2), dict=True)
sol

[]

In [3]:
# alpha = 1
# best response formulation
vi = sp.symbols('v_i')
bi = sp.symbols('b_i')
o  = sp.symbols('o')

qi = vi*bi/(bi+o) - bi

In [4]:
qi

b_i*v_i/(b_i + o) - b_i

In [39]:
sp.diff(qi, bi)

o*v_i/(b_i + o)**2 - 1

In [48]:
# alpha != 1
# best response formulation
vi = sp.symbols('v_i')
bi = sp.symbols('b_i')
o  = sp.symbols('o')
a  = sp.symbols('alpha')

qi = vi*bi**a/(bi**a+o) - bi

In [49]:
sp.print_latex(qi)
qi

- b_{i} + \frac{b_{i}^{\alpha} v_{i}}{b_{i}^{\alpha} + o}


-b_i + b_i**alpha*v_i/(b_i**alpha + o)

In [50]:
sp.print_latex(sp.diff(qi, bi))
sp.diff(qi, bi)

- \frac{\alpha b_{i}^{2 \alpha} v_{i}}{b_{i} \left(b_{i}^{\alpha} + o\right)^{2}} + \frac{\alpha b_{i}^{\alpha} v_{i}}{b_{i} \left(b_{i}^{\alpha} + o\right)} - 1


-alpha*b_i**(2*alpha)*v_i/(b_i*(b_i**alpha + o)**2) + alpha*b_i**alpha*v_i/(b_i*(b_i**alpha + o)) - 1

In [ ]:
vi = sp.symbols('v_i')
bi = sp.symbols('b_i')
o  = sp.symbols('o')
a  = sp.symbols('alpha')

qi = vi*bi**a/(bi**a+o) - bi

In [112]:
sp.expand((a-1)*xi**(-1/a) - (2*a-1)*xi**(1-1/a))

-2*alpha*x_i**(1 - 1/alpha) + alpha/x_i**(1/alpha) + x_i**(1 - 1/alpha) - 1/x_i**(1/alpha)

In [109]:
sp.diff(sp.expand((a-1)*xi**(-1/a) - (2*a-1)*xi**(1-1/a)), xi)

-2*alpha*x_i**(1 - 1/alpha)*(1 - 1/alpha)/x_i + x_i**(1 - 1/alpha)*(1 - 1/alpha)/x_i - 1/(x_i*x_i**(1/alpha)) + 1/(alpha*x_i*x_i**(1/alpha))

In [94]:
print_latex(sp.diff(xi**(1-1/a)-xi**(2-1/a),xi))
sp.diff(a*xi**(1-1/a)-a*xi**(2-1/a),xi)

\frac{x_{i}^{1 - \frac{1}{\alpha}} \left(1 - \frac{1}{\alpha}\right)}{x_{i}} - \frac{x_{i}^{2 - \frac{1}{\alpha}} \left(2 - \frac{1}{\alpha}\right)}{x_{i}}


alpha*x_i**(1 - 1/alpha)*(1 - 1/alpha)/x_i - alpha*x_i**(2 - 1/alpha)*(2 - 1/alpha)/x_i

In [96]:
sp.integrate(a*xi**(1-1/a)-a*xi**(2-1/a), xi)

alpha*Piecewise((x_i**(2 - 1/alpha)/(2 - 1/alpha), Ne(1/alpha, 2)), (log(x_i), True)) - alpha*Piecewise((x_i**(3 - 1/alpha)/(3 - 1/alpha), Ne(1/alpha, 3)), (log(x_i), True))

In [81]:
xi, a = sp.symbols('x_i alpha')
u     = sp.Function(x1)
sp.diff(xi**(1-1/a)*(1-xi), xi)

-x_i**(1 - 1/alpha) + x_i**(1 - 1/alpha)*(1 - 1/alpha)*(1 - x_i)/x_i

# Solving tullock as optimization problem with alpha=1 using potentials

In [ ]:
# max sum_i v_i*x_i (1-x_i/2). subj to. sum_i x_i = 1, x_i >= 0

In [20]:
p1+p2

Expression(UNKNOWN, UNKNOWN, ())

In [25]:
x1, x2 = cp.Variable(), cp.Variable()
v1, v2 = 4, 2
p1 = v1*x1 * (1 - x1/2)
p2 = v2*x2 * (1 - x2/2)
objective = cp.Maximize(p1+p2)
constraints = [x1+x2==1, x1>=0, x2>=0]
prob = cp.Problem(objective, constraints)
prob.solve()

DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP. Its following subexpressions are not:
4.0 @ var371 @ (1.0 + -var371 / 2.0)
2.0 @ var372 @ (1.0 + -var372 / 2.0)

In [33]:
x1, x2 = cp.Variable(), cp.Variable()
v1, v2 = 4, 2
p1 = v1 * x1 * (1 - x1 / 2)
p2 = v2 * (1-x1) * (1 - (1-x1) / 2)
objective = cp.Minimize(-p1 - p2)  # Minimize the negative of the original objective
constraints = [x1 >= 0]
prob = cp.Problem(objective, constraints)
prob.solve()

print("x1:", x1.value)
print("x2:", x2.value)
print("Maximum Profit:", p1.value + p2.value)

DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP. Its following subexpressions are not:
4.0 @ var563 @ (1.0 + -var563 / 2.0)
2.0 @ (1.0 + -var563) @ (1.0 + -(1.0 + -var563) / 2.0)

In [13]:
x = cp.Variable()
objective = cp.Maximize(cp.sum_squares(A @ x - b))
constraints = [0 <= x, x <= 1]
prob = cp.Problem(objective, constraints)

NameError: name 'A' is not defined